In [1]:
import pandas as pd
import math
from scipy import sparse

pd.set_option("display.max_rows", 60)
pd.set_option("display.min_rows", 30)

In [2]:
NUM_FEATURES = 5000
NUM_LABELS = 3993

In [3]:
df = pd.read_csv("data/train.csv", index_col=0)
df = df.fillna("")
df

,labels,features
ex_id,,
0,"446,521,1149,1249,1265,1482",0:0.084556 1:0.138594 2:0.094304 3:0.195764 4:...
1,"78,80,85,86",0:0.050734 1:0.762265 2:0.754431 3:0.065255 4:...
2,"457,577,579,640,939,1158",0:0.101468 1:0.138594 2:0.377215 3:0.130509 4:...
3,"172,654,693,1704",0:0.186024 1:0.346484 2:0.141456 3:0.195764 4:...
4,"403,508,1017,1052,1731,3183",0:0.135290 1:0.277187 2:0.141456 3:0.065255 4:...
5,"174,379,380,381,449,493,677,1328",0:0.101468 1:0.450429 2:0.141456 3:0.195764 4:...
6,"592,595,617,694,923",0:0.084556 1:1.628475 2:0.330063 3:0.783057 4:...
7,"446,499,500,671,804,1210",0:0.050734 1:0.866210 2:0.330063 3:0.195764 5:...
8,"64,180,611",0:0.050734 1:0.277187 2:0.188608 3:0.195764 4:...


In [4]:
for i, r in df.iterrows():
    if ":" in r["labels"]:  # incorrectly stored (no correct labels)
        print(i)
        
        # add misplaced "labels" string to beginning of "features"
        df.at[i, "features"] = r["labels"] + " " + r["features"]
        
        # make labels blank
        df.at[i, "labels"] = ""

93
252
509
1527
1939
1953
4029
4426
4643
4727
5231
5762
6295
6333
6703
7083
9477
9675
9999
10488
10736
10910
11674
12280
12599
13147
14167
14722


In [5]:
rows = []
for i, r in df.iterrows():
    row = [0 for i in range(NUM_FEATURES + NUM_LABELS)]
    
    # get feature dict
    fList = r["features"].split(" ")
    fDict = {}
    for f in fList:
        k, v = f.split(":")
        row[int(k)] = float(v)
    
    # get label list
    lList = r["labels"].split(",")
    for l in lList:
        if l:
            row[NUM_FEATURES + int(l)] = 1
    
    rows.append(row)

df_clean = pd.DataFrame(rows, columns=["f_{}".format(i) for i in range(NUM_FEATURES)]+["l_{}".format(i) for i in range(NUM_LABELS)])
df_clean

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,l_3983,l_3984,l_3985,l_3986,l_3987,l_3988,l_3989,l_3990,l_3991,l_3992
0,0.084556,0.138594,0.094304,0.195764,0.612552,0.106491,0.137765,0.145839,0.304610,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,0.050734,0.762265,0.754431,0.065255,0.350030,0.000000,0.137765,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,0.101468,0.138594,0.377215,0.130509,0.175015,0.745434,0.137765,1.020873,1.218441,0.309297,...,0,0,0,0,0,0,0,0,0,0
3,0.186024,0.346484,0.141456,0.195764,0.262522,0.425962,0.551062,0.437517,0.609220,0.618594,...,0,0,0,0,0,0,0,0,0,0
4,0.135290,0.277187,0.141456,0.065255,1.137597,0.106491,0.137765,0.583356,0.152305,0.154649,...,0,0,0,0,0,0,0,0,0,0
5,0.101468,0.450429,0.141456,0.195764,0.350030,0.851925,0.275531,0.145839,0.304610,1.237188,...,0,0,0,0,0,0,0,0,0,0
6,0.084556,1.628475,0.330063,0.783057,0.350030,2.023321,0.826593,1.020873,9.899832,0.618594,...,0,0,0,0,0,0,0,0,0,0
7,0.050734,0.866210,0.330063,0.195764,0.000000,3.088227,0.275531,0.291678,0.304610,0.618594,...,0,0,0,0,0,0,0,0,0,0
8,0.050734,0.277187,0.188608,0.195764,0.262522,0.745434,0.275531,0.291678,0.456915,0.618594,...,0,0,0,0,0,0,0,0,0,0
9,0.355137,2.494685,0.660127,0.913566,0.612552,0.319472,0.413296,1.312552,29.394884,0.927891,...,0,0,0,0,0,0,0,0,0,0


In [6]:
data_sparse = sparse.coo_matrix(df_clean.values)

In [7]:
sparse.save_npz("data/data_cleaned.npz", data_sparse)